In [49]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from keras import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications import VGG19,Xception
from tensorflow.keras.layers import Input,Flatten,Dense,BatchNormalization,Activation,Dropout,GlobalAveragePooling2D,MaxPooling2D,RandomFlip,RandomZoom,RandomRotation
from keras.datasets import cifar10

In [50]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [51]:
base_model = Xception(include_top=False, weights='imagenet', input_shape=(224,224,3), classes=y_train.shape[1])
base_model.trainable = False

In [52]:
inputs = tf.keras.Input(shape=(32, 32, 3))
x = tf.keras.layers.Lambda(lambda image: tf.image.resize(image, (224,224)))(inputs)
x = tf.keras.applications.xception.preprocess_input(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(10, activation=('softmax'))(x)
model = tf.keras.Model(inputs, outputs)

In [53]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [46]:
epochs = 10
batch_size = 32
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/10
1563/1563 [==============================] - 196s 124ms/step - loss: 0.4798 - accuracy: 0.8400
Epoch 2/10
1563/1563 [==============================] - 191s 122ms/step - loss: 0.3829 - accuracy: 0.8698
Epoch 3/10
1563/1563 [==============================] - 191s 122ms/step - loss: 0.3621 - accuracy: 0.8759
Epoch 4/10
1563/1563 [==============================] - 192s 123ms/step - loss: 0.3530 - accuracy: 0.8784
Epoch 5/10
1563/1563 [==============================] - 190s 122ms/step - loss: 0.3492 - accuracy: 0.8796
Epoch 6/10
1563/1563 [==============================] - 190s 122ms/step - loss: 0.3436 - accuracy: 0.8805
Epoch 7/10
1563/1563 [==============================] - 190s 122ms/step - loss: 0.3371 - accuracy: 0.8837
Epoch 8/10
1563/1563 [==============================] - 190s 122ms/step - loss: 0.3349 - accuracy: 0.8832
Epoch 9/10
1563/1563 [==============================] - 190s 122ms/step - loss: 0.3340 - accuracy: 0.8837
Epoch 10/10
1563/1563 [=======================

In [47]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

313/313 - 39s - loss: 0.3755 - accuracy: 0.8758 - 39s/epoch - 124ms/step

Test accuracy: 0.8758000135421753


In [48]:
model.save('xception_cifar.h5')